# License

In [1]:
import requests, bs4
from datetime import datetime
res = requests.get('https://opensource.org/licenses/mit-license.php')
res.raise_for_status()
soup = bs4.BeautifulSoup(res.text, "html.parser")
soup = soup.find_all(class_="field-item even")[0].find_all('p')
print('Copyright (C) ' +str(datetime.now().year) + ' computational-sediment-hyd https://github.com/computational-sediment-hyd')
print('Released under the MIT license'+'\n')
print( str(soup[3])[3:-4] +'\n\n' +  str(soup[4])[3:-4] +'\n\n' + str(soup[5])[3:-4] )

Copyright (C) 2018 computational-sediment-hyd https://github.com/computational-sediment-hyd
Released under the MIT license

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT.  IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER

# import module

In [2]:
import numpy as np
import pandas as pd
from numba.decorators import jit
import sys
import json
import glob as glob

# READ ME


- simulation model for 1 dimensional bed variation  of  bed load and sediment mixtures

- details of calculation condtion (in japanese) : 

In [3]:
screenclass = np.array( [ 2.0**0, 2.0**1, 2.0**2, 2.0**3, 2.0**4, 2.0**5, 2.0**6] )
dsize = np.array( [ 10**(0.5*(np.log10(screenclass[i]) + np.log10(screenclass[i+1]))) for i in range(len(screenclass)-1) ] )

h=1.0
ib = 1.0/200.0
hini = 1.0
n = 0.03
v = ib**0.5 * h**(2/3) / n
fr = v/np.sqrt(9.8*h)
print('Froude Number:' + str(fr))

columns = ['diameter[mm]' ]
df = pd.DataFrame(dsize, columns=columns)
df['tau_star'] = h*ib/dsize/1.65*1000
df

Froude Number:0.7529232524210427


,diameter[mm],tau_star
0,1.414214,2.142748
1,2.828427,1.071374
2,5.656854,0.535687
3,11.313708,0.267843
4,22.627417,0.133922
5,45.254834,0.066961


# Governing Equation of River flow 

\begin{align}
    \frac{\partial A}{\partial t}+\frac{\partial Q}{\partial x} = 0 \\
\end{align}

\begin{align}
    \frac{\partial Q}{\partial t}+\frac{\partial}{\partial x}\left(\frac{ Q^2}{ A}\right) + gA\frac{\partial H}{\partial x}+gAi_e = 0
\end{align}

In [4]:
# @jit
def flowmodel(A, Q, Adown, Qup, dzb, zb, dx, dt, g, n):
    B = 1.0
    imax = len(A)
    Anew, Qnew = np.zeros(imax), np.zeros(imax)
    
# continuous equation
    for i in range(1, imax-1): Anew[i] = A[i] - dt * ( Q[i] - Q[i-1] ) / dx
    Anew[0], Anew[-1] = Anew[1], Adown
    
# moumentum equation
    for i in range(1, imax-1): 
        ip, ic, im = (i+1, i, i-1) 
        Cr1 = 0.5*( Q[ic]/A[ic] + Q[ip]/A[ip] )*dt/dx
        Cr2 = 0.5*( Q[ic]/A[ic] + Q[im]/A[im] )*dt/dx
        dHdx1 = ( Anew[ip]/B + zb[ip] + dzb[ip] - Anew[ic]/B - zb[ic] - dzb[ic] ) / dx
        dHdx2 = ( Anew[ic]/B + zb[ic] + dzb[ic] - Anew[im]/B - zb[im] - dzb[im] ) / dx
        dHdx = (1.0 - Cr1) * dHdx1 + Cr2 * dHdx2
        
        Qnew[ic] = Q[ic] - dt * ( Q[ic]**2/A[ic] - Q[im]**2/A[im] ) / dx  \
                   - dt * g * Anew[ic] * dHdx \
                   - dt * g * A[ic] * n**2 * Q[ic]**2 / B**2 / ( A[ic]/B )**(10/3)
 
    Qnew[0], Qnew[-1] = Qup, Qnew[-2]
            
    return Anew, Qnew 

# Governing Equation of River Bed : Hirano model


\begin{align}
    (1-\lambda)\frac{\partial A_{zb}}{\partial t}+\frac{\partial }{\partial x} \sum_{i=1}^n ( Q_{bi}P_i)= 0 \\
\end{align}

\begin{align}
    \frac{\partial P_i}{\partial t} &= - \frac{1}{(1-\lambda)E_dB} \frac{\partial (Q_{bi}P_i)}{\partial x} - \frac{P_{si}}{E_d}\frac{\partial z_b}{\partial t} \\
    &= - \frac{1}{E_d}(\frac{\partial z_{bi}}{\partial t} + P_{si}\frac{\partial z_b}{\partial t}) 
\end{align}

\begin{align}
&P_{si} = 
\left\{ \begin{array}{ll}
    P_i \mbox{ in exchange layer} & (\frac{\partial z_b}{\partial t} > 0 ) \\
    P_i \mbox{ under exchange layer} & (\frac{\partial z_b}{\partial t} < 0 ) \\
\end{array} \right. \\
\end{align}


$Q_{bi}$ : Ashida-Michiue Eq. and  Egiazaroff Eq.

$E_d$ : thickness of exchange layer

\begin{align}
    \frac{\tau_{*c i}}{\tau_{*cm}} &= 0.85 \frac{D_m}{D_i} \qquad &(\frac{D_i}{D_m} < 0.4) \\
    \frac{\tau_{*ci}}{\tau_{*cm}} &= \left( \frac{ \log_e 19 }{ \log_e (19\frac{D_i}{D_m}) } \right)^2 \qquad &(\frac{D_i}{D_m} \geq 0.4)
\end{align}

In [5]:
# @jit
def sedimentmodel(dzb, dratio, A, Q, dx, dt, g, n, dsize, dratioStandard, hExlayer, qbup = None ):
    rhosw = 1.65 # grain specific gravity in water
    porosity = 0.4
    B = 1.0
    tscAve = 0.05 # critical tractive force of average grain size
    
    dzbnew = np.zeros_like( dzb )
    drationew = np.zeros_like( dratio )
    qbsub = np.zeros_like( dratio )
    dzbsub = np.zeros_like( dratio )
    
    for i, (Ap, Qp, dr) in enumerate( zip(A, Q, dratio) ):
        
        dAve = np.sum(dsize * dr)
        us = np.sqrt(g * Ap/B) * Qp/Ap * n / (Ap/B)**(2/3)
        tsAve = us**2.0/rhosw/g/dAve
        use = Qp/Ap / ( 6.0 + 2.5 * np.log( Ap/B/dAve/( 1.0+2.0*tsAve) ) )
        
        Kc=1.0
    
        for l, (dri, dsi) in enumerate(zip(dr, dsize)) :
            ts  = us**2.0 /rhosw/g/dsi
            tse = use**2.0/rhosw/g/dsi
            
# Egiazaroff Eq. 
            x = dsi/dAve
            tscbytscm = (np.log(19)/np.log(19*x))**2 if x > 0.4 else 0.85/x
            tsc = Kc * tscbytscm * tscAve 
        
            if ts > tsc :
# Ashida-Michiue Eq. 
                qbsub[i][l] = np.sqrt(rhosw * g* dsi**3.0) \
                        * 17.0 * tse**1.5 * ( 1.0 - tsc / ts) * ( 1.0 - np.sqrt(tsc / ts) ) \
                        * dri
            else:
                qbsub[i][l] = 0.0
    
    for i, qbs in enumerate(qbsub):
        if i == 0:
            if qbup is None : # equilibrium sand supply
                dzbsub[i][:] = 0.0
            else:
                for l, qbsi in enumerate(qbs):
                    dzbsub[i][l] = - dt / (1.0 - porosity) * ( qbsub[i][l]-qbup[l] )/dx
                    if np.abs(dzbsub[i][l]) > hExlayer : print('index = ' + str(i) + 'dzsub over error')
        else:
            for l, qbsi in enumerate(qbs):
                dzbsub[i][l] = - dt / (1.0 - porosity) * ( qbsub[i][l]-qbsub[i-1][l] )/dx
                if np.abs(dzbsub[i][l]) > hExlayer : print('index = ' + str(i) + 'dzsub over error')
                
    for i, qbs in enumerate(qbsub):
# update dzb 
        dzball = np.sum( dzbsub[i] )
        if np.abs(dzball) > hExlayer : print('index = ' + str(i) + 'dz over error')
            
        dzbnew[i] = dzb[i] + dzball
        
# update dratio
        dratioIn = dratioStandard[:] if dzball < 0.0 else dratio[i][:]
        drationew[i][:] = dratio[i][:] + (dzbsub[i][:] - dzball * dratioIn[:] ) / hExlayer        
    
        drationew[i] = np.where( drationew[i] > 0, drationew[i], 0)
        if np.min(drationew[i][:]) < 0 : print('index = ' + str(i) + 'ratio minus error') 
            
        # correct ration so that sum of ration become 100% 
        sumdr  =  np.sum( drationew[i] )
        drationew[i][:] /= sumdr
        
    return dzbnew, drationew, qbsub

# main

In [6]:
%%time
length = 5000.0
dx = 100.0
imax = int(length/dx)
dt = 10.0
totalTime = 100.0*3600.0
outTimeStep = 2.0*3600
g = 9.8
B = 1.0
hini = 1.0
n  = 0.03
ib = 1.0/200.0
ib2 = 1.0/1000.0

# grain diameter classification
screenclass = np.array( [ 2.0**0, 2.0**1, 2.0**2, 2.0**3, 2.0**4, 2.0**5, 2.0**6] )/1000
dsize = np.array( [ 10**(0.5*(np.log10(screenclass[i]) + np.log10(screenclass[i+1]))) for i in range(len(screenclass)-1) ] )

# percentage of grain size under exchange layer
dratioStandard = np.full_like(dsize, 1.0/len(dsize))

# initial percentage of grain size in exchange layer
dratio = np.full( (imax,len(dsize) ), dratioStandard )

# thickness of exchange layer 
hExlayer = dsize[-1]

# Initial & Boundary condition
A = np.full(imax, hini*B)
Q = ib**0.5*(A/B)**(5.0/3.0)/n*B #normal flow
Qup = Q[0]
dzb = np.zeros(imax)
zb = np.zeros(imax)
for i in range(1,imax):
    zb[i] = zb[i-1] + ib2*dx if i < 25 else zb[i-1] + ib*dx
zb = zb[::-1]

# for i, (Ap, zbp) in enumerate(zip(A, zb)):
#     if( Ap/B + zbp < WLdown) : A[i] = WLdown - zbp

def calAdown(Q, zb, dzb, ib):
    return (n**2*Q**2/ib/B**2)**0.3*B
        
# run-up calculation
timeRunUp = 3600 * 3.0
for i in range(int(timeRunUp/dt)):
    Adown = calAdown(Q[-1], zb[-1], dzb[-1], ib)
    A, Q = flowmodel(A, Q, Adown, Qup, dzb, zb, dx, dt, g, n)

# main calculation
for i in range(int(totalTime/dt)):
# cal bed variation
    dzb, dratio, qbsub = sedimentmodel(dzb, dratio, A, Q, dx, dt, g, n, dsize, dratioStandard, hExlayer )
# cal water profile
    ib = ( ( dzb[-2] + zb[-2] ) - ( dzb[-1] + zb[-1] ) )/dx
    Adown = calAdown(Q[-1], zb[-1], dzb[-1], ib)
    A, Q = flowmodel(A, Q, Adown, Qup, dzb, zb, dx, dt, g, n)
    
    # output 
    if( int(i*dt) % int(outTimeStep) ) == 0 :
        print(str( int(i*dt) ) + ' second') 
        profile = []
        for ap, z, r, q in zip(A, dzb, dratio, qbsub): profile.append({'A':ap, 'dzb':z, 'ratio':list(r), 'qb':list(q)})
        json.dump( {'time':i*dt, 'data':profile}, open('%010d' % int(i*dt) + 'sec.json', 'w') )
            
# join json files
d = [ json.load( open(f, 'r') ) for f in glob.glob('*sec.json') ]
d.sort(key=lambda x: x['time'])
json.dump( d, open('case01.json', 'w') )

0 second
7200 second
14400 second
21600 second
28800 second
36000 second
43200 second
50400 second
57600 second
64800 second
72000 second
79200 second
86400 second
93600 second
100800 second
108000 second
115200 second
122400 second
129600 second
136800 second
144000 second
151200 second
158400 second
165600 second
172800 second
180000 second
187200 second
194400 second
201600 second
208800 second
216000 second
223200 second
230400 second
237600 second
244800 second
252000 second
259200 second
266400 second
273600 second
280800 second
288000 second
295200 second
302400 second
309600 second
316800 second
324000 second
331200 second
338400 second
345600 second
352800 second
Wall time: 5min 5s
